Author: Junsheng Pei
    Zihao Yang
    Nuochen Lyu

<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [4]:
# Model definition
# Derived from models in `https://github.com/kuangliu/pytorch-cifar`

In [166]:
#!/usr/bin/env python

import sys
import argparse
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
import pandas as pd

from sklearn import linear_model


In [212]:
# --
# CLI
# The default values of most arguments are sufficient for this problem
# You may want to use --subset-train or --subset-val to use only a subset
# of the data while testing your code.
# specific location

#### This model ran quite slow in my jupyter-notebook, so I set the epochs to 1. If you want to test it, please
####copy all the code into .py and ran it in terminal.
epochs =23
subset_val = None
batch_size = 128
subset_train = None
gpu = False
seed = 123
classes = ('airplanes','bear','blimp','comet','crab','dog','dolphin''giraffe',
'goat','gorilla','kangaroo','killer-whale','leopards','llama','penguin','porcupine','teddy-bear',
'triceratops','unicorn','zebra')

In [218]:
# Loading data
def load_data():
    print('cifar10.py: making dataloaders...', file=sys.stderr)
    # transforms define preprocessing on the dataset
    transform_train = transforms.Compose([
#         # random crop
        transforms.RandomCrop(64),
#         #  random horizontal flip
         transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std= (0.24705882352941178, 0.24352941176470588, 0.2615686274509804))
    ])
    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std= (0.24705882352941178, 0.24352941176470588, 0.2615686274509804))
    ])
    # loading the dataset
    try:
#         trainset = datasets.CIFAR10(root=args.data_dir, train=True, download=True, transform=transform_train)
#         valset  = datasets.CIFAR10(root=args.data_dir, train=False, download=True, transform=transform_val)
        dataset = datasets.ImageFolder(root='./20_categories_training',transform=transform_train)
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        trainset, valset = torch.utils.data.random_split(dataset, [train_size, val_size])
    except:
        raise Exception('error loading data -- try rerunning w/ `--download` flag')
    # selecting a random subset of the dataset if requested
    np.random.seed(seed)
    if (subset_train is not None):
        idxs = np.random.choice(len(trainset), subset_train, replace=False)
        trainset = torch.utils.data.Subset(trainset, idxs)
    if (subset_val is not None):
        idxs = np.random.choice(len(valset), subset_val, replace=False)
        trainset = torch.utils.data.Subset(valset, idxs)
    # data loading objects
    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True,
    )
    valloader = torch.utils.data.DataLoader(
        valset,
        batch_size=512,
        shuffle=False,
        pin_memory=True,
    )
    dataloaders = {
        "train" : trainloader,
        "val"  : valloader,
    }
    return dataloaders

dataloaders = load_data()

cifar10.py: making dataloaders...


In [217]:
# --
# Model definition
# Derived from models in `https://github.com/kuangliu/pytorch-cifar`
class PreActBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.bn1   = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)

        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        return out + shortcut


class ResNet18(nn.Module):
    def __init__(self, num_blocks=[2, 2, 2, 2], num_classes=20):
        super().__init__()
        self.in_channels = 64

        self.prep = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layers = nn.Sequential(
            self._make_layer(64, 64, num_blocks[0], stride=1),
            self._make_layer(64, 128, num_blocks[1], stride=2),
            self._make_layer(128, 256, num_blocks[2], stride=2),
            self._make_layer(256, 256, num_blocks[3], stride=2),
        )

        self.classifier = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):

        strides = [stride] + [1] * (num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(PreActBlock(in_channels=in_channels, out_channels=out_channels, stride=stride))
            in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.float()
        x = self.prep(x)
        x = self.layers(x)

        x_avg = F.adaptive_avg_pool2d(x, (1, 1))
        x_avg = x_avg.view(x_avg.size(0), -1)

        x_max = F.adaptive_max_pool2d(x, (1, 1))
        x_max = x_max.view(x_max.size(0), -1)

        x = torch.cat([x_avg, x_max], dim=-1)

        x = self.classifier(x)

        return x

# --
# Learning rate functions
def set_lr(lr, optimizer):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#lr = calc_lr(lr, epoch, i, N)
def const_lr_maker(const=0.001):
    # TO IMPLEMENT Part (e)
    def f(curr_lr, epoch, batch_idx, N):
        return const
    return f

def anneal_lr_maker(lr=0.1, factor=0.99):
    # TO IMPLEMENT Part (e)
    def f(curr_lr, epoch, batch_idx, N):
        return curr_lr * factor
    return f

def special_lr_maker(hp_max=0.1, epochs=3, hp_init=0.0, hp_final=0.005, extra=5):
    def f(curr_lr, epoch, batch_idx, N):
        progress = epoch + batch_idx / N
        if progress < epochs / 2:
            return 2 * hp_max * (1 - (epochs - progress) / epochs)
        elif progress <= epochs:
            return hp_final + 2 * (hp_max - hp_final) * (epochs - progress) / epochs
        elif progress <= epochs + extra:
            return hp_final * (extra - (progress - epochs)) / extra
        else:
            return hp_final / 10
    return f

const_lr = const_lr_maker()
anneal_lr = anneal_lr_maker()
special_lr = special_lr_maker(epochs=epochs+3)

# --
# Model training
def train_with_lr_scheme(calc_lr):

    print('cifar10.py: initializing model...', file=sys.stderr)
    if (torch.cuda.is_available() and gpu):
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    model = ResNet18().to(device)
    model.verbose = True
    # --
    # Initialize optimizer
    print('cifar10.py: training...', file=sys.stderr)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    lr = 0.01

    trainloader, valloader = dataloaders['train'], dataloaders['val']
    N = len(trainloader)
    iter_count = 0 # a count of all iterations
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        total_train = 0; correct_train = 0
        total_val = 0; correct_val = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()

            if (torch.cuda.is_available() and gpu):
                labels = labels.cuda()
                inputs = inputs.cuda()
            # forward + backward
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # optimize
            lr = calc_lr(lr, epoch, i, N)
            set_lr(lr, optimizer)
            iter_count += 1
            optimizer.step()

            # compute training statistics
            logits = outputs.cpu().detach().numpy()
            y_pred_train = np.argmax(logits, axis=1)
            y_train = labels.cpu().detach().numpy()
            total_train += y_train.shape[0]
            correct_train += sum(y_pred_train == y_train)
            running_loss += loss.item()
            print("Epoch:", epoch, "\tMiniBatch:", i, "\tPartial Training Accuracy:", correct_train/total_train,  "\tRunning Loss:", running_loss/(i+1))
        print("Epoch:", epoch, "\tFinal Training Accuracy:", {correct_train/total_train})
        for i, data in enumerate(valloader, 0):
            # get the inputs
            inputs, labels = data
            if (torch.cuda.is_available() and gpu):
                labels = labels.cuda()
                inputs = inputs.cuda()
            # predict outputs
            outputs = model(inputs)
            logits = outputs.cpu().detach().numpy()

            # compute validation statistics
            y_pred_train = np.argmax(logits, axis=1)
            y_train = labels.cpu().detach().numpy()
            total_val += y_train.shape[0]
            correct_val  += sum(y_pred_train == y_train)
            print("Epoch:", epoch, "\tMiniBatch:", i, "\tPartial Validation Accuracy:", correct_val/total_val)
        print("Epoch:", epoch, "\tFinal Validation Accuracy:", {correct_val/total_val})
        result = {}
        result['train_accuracy'] = correct_train/total_train
        result['val_accuracy'] = correct_val/total_val
        result['num_epochs'] = epochs
        result['train_loss'] = running_loss
    return result

In [1]:
print("===== Training network with an annealing LR scheme ====")
result_anneal = train_with_lr_scheme(anneal_lr)
result_anneal['lr_scheme'] = 'anneal'
print("===== Training network with a special LR scheme ====")
result_special = train_with_lr_scheme(special_lr)
result_special['lr_scheme'] = 'special_0'
print("==== Summary ===")
df = pd.DataFrame([result_const, result_anneal, result_special])
print(df)

I run the code in terminal and have the final reults below:

#Epoch: 19   MiniBatch: 8    Partial Training Accuracy: 0.3585069444444444   Running Loss: 2.0570848253038196
#Epoch: 19   MiniBatch: 9    Partial Training Accuracy: 0.3591666666666667   Running Loss: 2.0550736665725706
#Epoch: 19   Final Training Accuracy: {0.3591666666666667}
#Epoch: 19   MiniBatch: 0    Partial Validation Accuracy: 0.37209302325581395
#Epoch: 19   Final Validation Accuracy: {0.37209302325581395}

### The validation accuracy is 37.2